In [167]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [168]:
!pip install nltk
!pip install transformers
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [169]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [170]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# 1. 데이터 불러오기

In [171]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/study/labeling_1.csv')
df1.head()

,column_num,sentence,index,overall,Unnamed: 4
0,0,Would be super sexy if you are only about 4 ft...,0,1,1.0
1,0,"I'm 5'5"" and this barely stretched to the unde...",0,1,1.0
2,0,Don't waste your $,0,1,3.0
3,1,Tiny fit and terrible print!,1,1,3.0
4,1,Got it for my brother who normally wears a large.,1,1,1.0


In [172]:
df1.rename(columns={'Unnamed: 4': 'cat'}, inplace=True)

In [173]:
df1 = df1[['sentence', 'cat']]

In [174]:
df1.head()

,sentence,cat
0,Would be super sexy if you are only about 4 ft...,1.0
1,"I'm 5'5"" and this barely stretched to the unde...",1.0
2,Don't waste your $,3.0
3,Tiny fit and terrible print!,3.0
4,Got it for my brother who normally wears a large.,1.0


In [175]:
df1.cat.unique()

array([ 1.,  3.,  0., -1.,  2., 69., 70., 71., 72., nan])

In [176]:
df1 = df1[df1.cat < 10]

In [177]:
df1.cat = df1.cat.astype('int')

<ipython-input-177-0813ea14060c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.cat = df1.cat.astype('int')


In [178]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581 entries, 0 to 588
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  581 non-null    object
 1   cat       581 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.6+ KB


In [179]:
df1 = df1.dropna()

In [180]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581 entries, 0 to 588
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  581 non-null    object
 1   cat       581 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.6+ KB


In [181]:
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/study/labeling_2.csv', index_col = 0)
df2.head()

,column_num,sentence,index,overall,cat
0,22000,"The material feels pretty cheap, not very soft.",22000,2,3
1,22000,Colors and pattern are pretty but the material...,22000,2,2
2,22001,"I guess the ""fits as described"" reviews were p...",22001,2,1
3,22001,"Seller was also reluctant to refund due to ""sh...",22001,2,0
4,22001,Still too expensive for a shirt that may never...,22001,2,3


In [182]:
df2 = df2[['sentence', 'cat']]

In [183]:
df2.head()

,sentence,cat
0,"The material feels pretty cheap, not very soft.",3
1,Colors and pattern are pretty but the material...,2
2,"I guess the ""fits as described"" reviews were p...",1
3,"Seller was also reluctant to refund due to ""sh...",0
4,Still too expensive for a shirt that may never...,3


In [184]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384 entries, 0 to 434
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  384 non-null    object
 1   cat       384 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.0+ KB


In [185]:
df3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/study/labeling_3.csv', index_col = 0)
df3.head()

,column_num,sentence,index,overall,cat
0,44000,Normalement o/s me fait mais l le matriel ne s...,44000,3,-1
1,44000,J'ai des cuisses normale (je porte mdium) mais...,44000,3,-1
2,44000,De bonne qualit et couleur rose vif.,44000,3,-1
3,44001,"Love this tank, but it is way way too small.",44001,3,1
4,44001,I usually wear a medium.,44001,3,1


In [186]:
df3 = df3[['sentence', 'cat']]

In [187]:
df3.head()

,sentence,cat
0,Normalement o/s me fait mais l le matriel ne s...,-1
1,J'ai des cuisses normale (je porte mdium) mais...,-1
2,De bonne qualit et couleur rose vif.,-1
3,"Love this tank, but it is way way too small.",1
4,I usually wear a medium.,1


In [188]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 449 entries, 0 to 448
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  449 non-null    object
 1   cat       449 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 10.5+ KB


In [189]:
df4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/study/labeling_4.csv', index_col = 0)
df4.head()

,column_num,sentence,index,overall,cat
0,66000,Just wish she'd won.,66000,4,-1
1,66002,I like them but I ordered the wrong size and I...,66002,4,1
2,66002,So I had to return them,66002,4,-1
3,66003,"Beautiful coat, just to small, the sizing deta...",66003,4,1
4,66003,even with the size chart is off.,66003,4,1


In [190]:
df4 = df4[['sentence', 'cat']]

In [191]:
df4.head()

,sentence,cat
0,Just wish she'd won.,-1
1,I like them but I ordered the wrong size and I...,1
2,So I had to return them,-1
3,"Beautiful coat, just to small, the sizing deta...",1
4,even with the size chart is off.,1


In [192]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452 entries, 0 to 451
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  452 non-null    object
 1   cat       452 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 10.6+ KB


In [193]:
df5 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/study/labeling_5.csv', encoding = 'CP949')
df5.head()

,column_num,sentence,index,overall,Unnamed: 4
0,88000,Has held up well after one international trip.,88000,5,아님
1,88002,"Fits well, looks great!",88002,5,사이즈
2,88004,"Love this top, it's fabulous, just buy it!",88004,5,아님
3,88005,I absolutely loved the shirt I got from here.,88005,5,아님
4,88005,It fit perfectly and it may have took longer t...,88005,5,사이즈


In [194]:
df5.rename(columns={'Unnamed: 4': 'cat'}, inplace=True)

In [195]:
df5 = df5[['sentence', 'cat']]

In [196]:
df5.head()

,sentence,cat
0,Has held up well after one international trip.,아님
1,"Fits well, looks great!",사이즈
2,"Love this top, it's fabulous, just buy it!",아님
3,I absolutely loved the shirt I got from here.,아님
4,It fit perfectly and it may have took longer t...,사이즈


In [197]:
df5.cat.unique()

array(['아님', '사이즈', '퀄리티', '배송', '색상', '배', 'ㅋ뤄', '쿠러', nan], dtype=object)

In [198]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39003 entries, 0 to 39002
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  39003 non-null  object
 1   cat       500 non-null    object
dtypes: object(2)
memory usage: 609.5+ KB


In [199]:
df5 = df5.dropna()

In [200]:
df5['cat'] = df5['cat'].apply(lambda x : '배송' if x == '배' else x)
df5['cat'] = df5['cat'].apply(lambda x : '퀄리티' if x == 'ㅋ뤄' else x)
df5['cat'] = df5['cat'].apply(lambda x : '퀄리티' if x == '쿠러' else x)

In [201]:
df5.cat.unique()

array(['아님', '사이즈', '퀄리티', '배송', '색상'], dtype=object)

In [202]:
df5.loc[df5['cat'] == '배송', 'cat'] = 0
df5.loc[df5['cat'] == '퀄리티', 'cat'] = 3
df5.loc[df5['cat'] == '아님', 'cat'] = -1
df5.loc[df5['cat'] == '사이즈', 'cat'] = 1
df5.loc[df5['cat'] == '색상', 'cat'] = 2

In [203]:
df5.cat.unique()

array([-1, 1, 3, 0, 2], dtype=object)

In [204]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  500 non-null    object
 1   cat       500 non-null    object
dtypes: object(2)
memory usage: 11.7+ KB


In [205]:
df = pd.concat([df1, df2])
df = pd.concat([df, df3])
df = pd.concat([df, df4])
df = pd.concat([df, df5])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2366 entries, 0 to 499
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  2366 non-null   object
 1   cat       2366 non-null   object
dtypes: object(2)
memory usage: 55.5+ KB


In [206]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [207]:
remove_df = pd.DataFrame({"col":[]})
remove_df.head()

,col


In [208]:
def find(sentence): 
  for i, sent in enumerate(sentence):
      input_ids = tokenizer.encode(sent, add_special_tokens=True)
      if len(input_ids) > 30:
        remove_df.loc[remove_df.index.max() + 1 if len(remove_df) > 0 else 0, "col"] = i

In [209]:
find(df.sentence)

In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2366 entries, 0 to 499
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  2366 non-null   object
 1   cat       2366 non-null   object
dtypes: object(2)
memory usage: 55.5+ KB


In [211]:
df = df[~df.index.isin(remove_df['col'])]

In [212]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2187 entries, 0 to 499
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  2187 non-null   object
 1   cat       2187 non-null   object
dtypes: object(2)
memory usage: 51.3+ KB


In [213]:
df.reset_index(drop=True, inplace = True)

In [214]:
df.tail()

,sentence,cat
2182,This would definitely run in the small/medium ...,1
2183,"It's a good bra, you just need to be aware tha...",1
2184,Used it until about halfway through my third t...,3
2185,"Will start using again, postpartum, soon!",-1
2186,I love this shirt!,3


In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2187 entries, 0 to 2186
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  2187 non-null   object
 1   cat       2187 non-null   object
dtypes: object(2)
memory usage: 34.3+ KB


In [216]:
'''from sklearn.model_selection import train_test_split
train_idx, val_idx = train_test_split(df.index, test_size=0.2, random_state=42)

# train 세트와 validation 세트를 분할합니다.
test = df.loc[val_idx]
df = df.loc[train_idx]'''

'from sklearn.model_selection import train_test_split\ntrain_idx, val_idx = train_test_split(df.index, test_size=0.2, random_state=42)\n\n# train 세트와 validation 세트를 분할합니다.\ntest = df.loc[val_idx]\ndf = df.loc[train_idx]'

# 2. 전처리


In [217]:
sw = stopwords.words('english')

In [218]:
def clean_text(text):
    
    text = text.lower() 
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
    text = re.sub(r",", " ",text) 
    html = re.compile(r'<.*?>') 
    text = html.sub(r'',text)
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') 
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = " ".join(text) 
    
    return text

In [219]:
df['sentence'] = df['sentence'].apply(lambda x: clean_text(x))

In [220]:
df['cat'] = df['cat'].apply(lambda x : 4 if x == -1 else x)

In [221]:
df.head()

,sentence,cat
0,would super sexy ft tall,1
1,barely stretched underside boobs,1
2,waste,3
3,tiny fit terrible print,3
4,got brother normally wears large,1


In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2187 entries, 0 to 2186
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  2187 non-null   object
 1   cat       2187 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.3+ KB


In [223]:
#df['sentence'] = df['sentence'].apply(lambda x: x if len(x.split(' ')) > 3 else np.nan)
#df = df.dropna()
#df.info()

In [224]:
df.cat.unique()

array([1, 3, 4, 2, 0])

In [225]:
train_sentence = df.sentence.values
train_labels = df.cat.values

# 3. 토큰화

In [226]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [227]:
max_len = 30

In [228]:
def MyTokenize(sentence):
  input_ids = []
  attention_masks = []

  for sent in sentence:
      encoded_dict = tokenizer.encode_plus(
                          sent,                      
                          add_special_tokens = True,
                          max_length = max_len,          
                          pad_to_max_length = True,
                          return_attention_mask = True,  
                          return_tensors = 'pt',   
                    )
        
      input_ids.append(encoded_dict['input_ids'])
      
      attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  
  attention_masks = torch.cat(attention_masks, dim=0)

  return input_ids, attention_masks

In [229]:
train_labels = torch.tensor(train_labels)

In [230]:
train_input_ids, train_attention_masks = MyTokenize(train_sentence)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# 4. 모델 학습

In [231]:
dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)

train_size = int(0.8 * len(dataset))

val_size = len(dataset)  - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

1,749 training samples
  438 validation samples


In [232]:
batch_size = 32

train_dataloader = DataLoader(
            train_dataset, 
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size
        )

In [233]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 5, 
    output_attentions = False, 
    output_hidden_states = False, 
)

model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [234]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8 
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [235]:
epochs = 10

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [236]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [237]:
def format_time(elapsed):
  
    elapsed_rounded = int(round((elapsed)))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [238]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

total_t0 = time.time()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
   
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)        
        loss = output.loss
        total_train_loss += loss.item()
      
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
       
        optimizer.step()
      
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
 
    print("")
    print("Running Validation...")
    t0 = time.time()
  
    model.eval()
 
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
  
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
      
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
   
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
   
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
   
    avg_val_loss = total_eval_loss / len(validation_dataloader)
  
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/study/MyBert_ipyb.ipynb')
        best_eval_accuracy = avg_val_accuracy
  
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 10 ========
Training...

  Average training loss: 1.30
  Training epcoh took: 0:00:11

Running Validation...
  Accuracy: 0.61

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.97
  Training epcoh took: 0:00:11

Running Validation...
  Accuracy: 0.65

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.80
  Training epcoh took: 0:00:10

Running Validation...
  Accuracy: 0.68

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.68
  Training epcoh took: 0:00:10

Running Validation...
  Accuracy: 0.67

======== Epoch 5 / 10 ========
Training...

  Average training loss: 0.56
  Training epcoh took: 0:00:10

Running Validation...
  Accuracy: 0.68

======== Epoch 6 / 10 ========
Training...

  Average training loss: 0.46
  Training epcoh took: 0:00:10

Running Validation...
  Accuracy: 0.65

======== Epoch 7 / 10 ========
Training...

  Average training loss: 0.38
  Training epcoh took: 0:00:10

Running Validatio

# 6. 모델 테스트

In [240]:
test_dataset  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/study/text_to_sentence.csv')
#test_dataset = test

In [241]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279057 entries, 0 to 279056
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   column_num  279057 non-null  int64 
 1   sentence    279057 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.3+ MB


In [242]:
test_dataset['sentence2'] = test_dataset['sentence'].apply(lambda x: clean_text(x))

In [245]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

remove_df = pd.DataFrame({"col":[]})

def find(sentence): 
  for i, sent in enumerate(sentence):
      input_ids = tokenizer.encode(sent, add_special_tokens=True)
      if len(input_ids) > 25:
        remove_df.loc[remove_df.index.max() + 1 if len(remove_df) > 0 else 0, "col"] = i

find(test_dataset.sentence)
test_dataset = test_dataset[~test_dataset.index.isin(remove_df['col'])]   

KeyboardInterrupt: ignored

In [244]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278485 entries, 0 to 279056
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   column_num  278485 non-null  int64 
 1   sentence    278485 non-null  object
 2   sentence2   278485 non-null  object
dtypes: int64(1), object(2)
memory usage: 8.5+ MB


In [246]:
test_sentence = test_dataset.sentence2

In [ ]:
#test_dataset['cat'] = test_dataset['cat'].apply(lambda x : 4 if x == -1 else x)

In [247]:
# 모델 인스턴스 생성
model2 = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 5, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# 저장된 가중치 불러오기
model2.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/study/my_bert_model.pth'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [248]:
test_input_ids, test_attention_masks = MyTokenize(test_sentence)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [249]:
test_tensor = TensorDataset(test_input_ids, test_attention_masks)

In [250]:
model2 = model2.to(device)

model2.eval()

test_dataloader = DataLoader(
    test_tensor,
    sampler = SequentialSampler(test_dataset),
    batch_size = 32
)

predictions = []

for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    
    with torch.no_grad():
        outputs = model2(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    
    predictions.extend(np.argmax(logits, axis=1).flatten())

test_dataset['label']  = list(predictions)


In [251]:
test_dataset.head()

,column_num,sentence,sentence2,label
0,0,Would be super sexy if you are only about 4 ft...,would super sexy ft tall,1
1,0,"I'm 5'5"" and this barely stretched to the unde...",barely stretched underside boobs,1
2,0,Don't waste your $,waste,3
3,1,Tiny fit and terrible print!,tiny fit terrible print,3
4,1,Got it for my brother who normally wears a large.,got brother normally wears large,1


In [ ]:
'''import matplotlib.pyplot as plt

# Creating the bar graph
width = 0
alpha = 0.5

data_cat = test_dataset.groupby(test_dataset.cat).count()
data_label = test_dataset.groupby(test_dataset.label).count()
plt.bar(data_cat.index - width/2, \
        data_cat.sentence, label='cat', alpha = 0.5)
plt.bar(data_label.index + width/2, \
        data_label.sentence, label='label', alpha = 0.5)


# Adding labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Bar Graph with Two Datasets')

# Adding legend
plt.legend()

# Displaying the graph
plt.show()'''

In [252]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278485 entries, 0 to 279056
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   column_num  278485 non-null  int64 
 1   sentence    278485 non-null  object
 2   sentence2   278485 non-null  object
 3   label       278485 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 10.6+ MB


In [ ]:
#wrong = test_dataset[test_dataset.cat != test_dataset.label]
#wrong.head()

In [253]:
test_dataset.to_csv('sentence_label.csv', index=False)